In [1]:
import pyiron_workflow.graph.gui
%config IPCompleter.evaluation='unsafe'

from pyiron_workflow.graph import gui, base

In [4]:
pf = gui.PyironFlow(['murnaghan2', 'energy', 'murn4', 'db']) # 'landau2',
pf.gui

In [3]:
xx

NameError: name 'xx' is not defined

In [ ]:
import pyiron_database.instance_database as idb



graph = pf.graph
graph = base.update_execution_graph(graph, debug=True)
nodeB = graph.nodes["BulkStaticEnergyF"].node
nodeI = graph.nodes["IterNode"].node

idb.get_hash(nodeB), id(nodeB), id(nodeI.inputs.node.value.node)



  

In [ ]:
id(nodeB.outputs.out.node), id(nodeB)

In [ ]:
nodeI.outputs.data.keys()

In [ ]:
nodeB.inputs

In [ ]:
nodeI.inputs.node.value

In [ ]:
idb.get_hash(nodeI.inputs.node.value.node)

In [ ]:
 nodeI.inputs.node.value.node.inputs

In [ ]:
base._find_input_nodes(base._remove_node_inputs(graph), 1)

In [ ]:
from pyiron_workflow.simple_workflow import extract_output_parameters_from_function, _return_as_macro_node

In [ ]:
import pyiron_nodes as pn

pn.local_workflows.test.BulkStaticEnergy

In [ ]:
from pyiron_workflow import as_macro_node, as_function_node, Node, Port

@as_function_node("energy")
def BulkStaticEnergy(name: str, a: float = None):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Bulk = pyiron_nodes.atomistic.structure.build.Bulk(name=name, a=a) 
    wf.M3GNet = pyiron_nodes.atomistic.engine.ase.M3GNet() 
    wf.Static = pyiron_nodes.atomistic.calculator.ase.Static(structure=wf.Bulk, engine=wf.M3GNet) 
    wf.GetEnergyLast = pyiron_nodes.atomistic.calculator.output.GetEnergyLast(calculator=wf.Static) 

    out = wf.GetEnergyLast.pull()

    return out

    # return wf.GetEnergyLast.outputs.energy_last

In [ ]:
@as_macro_node(["BulkStaticEnergy", "BulkStructure"])
def BulkStaticEnergy(name: str, a: float = None):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Bulk = pyiron_nodes.atomistic.structure.build.Bulk(name=name, a=a) 
    wf.M3GNet = pyiron_nodes.atomistic.engine.ase.M3GNet() 
    wf.Static = pyiron_nodes.atomistic.calculator.ase.Static(structure=wf.Bulk, engine=wf.M3GNet) 
    wf.GetEnergyLast = pyiron_nodes.atomistic.calculator.output.GetEnergyLast(calculator=wf.Static) 

    return wf.GetEnergyLast.outputs.energy_last, wf.Bulk

In [ ]:
args = ["BulkStaticEnergy", "BulkStructure"]
# print("args", args, isinstance(args[0], str))
if isinstance(args[0], str):
    output_labels = list(args)
else:
    output_labels = None
output_labels

In [ ]:
out = BulkStaticEnergy(name='Al').run()
out
#isinstance(out, Port)
#out.node._workflow.run()
# out._workflow.run()
# _return_as_macro_node(BulkStaticEnergy('Al'), 'Bulk', 'Energy', 'macro')

In [ ]:
pf.graph.nodes["subgraph"].node.node_type

In [ ]:
base.get_inputs_of_graph(pf.graph.nodes["subgraph"].graph, exclude_unconnected_default_ports=True)

In [ ]:
print(base.get_code_from_graph(pf.graph.nodes["subgraph"].graph))

In [ ]:
db

In [ ]:
pf.graph.nodes["ShowTable"].node.outputs.df.value.iloc[7].hash

In [ ]:
from pyiron_database.instance_database import restore_node_from_database, PostgreSQLInstanceDatabase
from sqlalchemy.orm import sessionmaker
import pandas as pd

user = 'joerg'
password = 'none'
host = 'localhost'
port = '5432'
database = 'joerg'

connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
db = PostgreSQLInstanceDatabase(connection_str)
db.drop()
db.init()



In [ ]:
Session = sessionmaker(bind=db.engine)
session = Session()

df = pd.read_sql(session.query(db.table).statement, session.bind)

session.close()

_, graph = restore_node_from_database(db=db, node_hash=df.hash.iloc[19])
#_, graph = restore_node_from_database(db=db, node_hash='cf0bda97885d0ddf6b8a4cd1a92d38806147e0721a9f646cfb3a37487dd6a7b2')
pyiron_workflow.graph.gui.GuiGraph(graph)

In [ ]:
df.query("hash == '9c051a26834ef83bbbe6c11c9fc0fc8c24cc06229432d31021fd00faa4c738ef'").iloc[0]

In [ ]:
df[df.hash==inputs["node"].split("@")[0]].iloc[0]

In [ ]:
button

In [ ]:
import pyiron_nodes as pn
from pyiron_workflow.simple_workflow import get_import_path_from_type

pn.atomistic.structure.build.Bulk('Al')._func.__qualname__

In [ ]:
import pandas as pd

df = pf.graph.nodes["IterNode"].node.outputs.df.value
state = df.__getstate__()
print(state)

df_new = pd.DataFrame() # .__setstate__(state)
df_new._mgr = state["_mgr"]
df_new._metadata = state["_metadata"]
df_new2 = pd.DataFrame()
df_new2.__setstate__(state)
df_new2

In [ ]:
node = pf.graph.nodes["IterNode"].node

In [ ]:
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.orm import sessionmaker

user = 'joerg'
password = 'none'
host = 'localhost'
port = '5432'
database = 'joerg'

connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_str)
engine.connect()

inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

In [ ]:
import pyiron_database.instance_database as idb

node = pf.graph.nodes["IterNode"].node

db = idb.PostgreSQLInstanceDatabase(connection_str)
db.drop()
db.init()
hash = idb.store_node_in_database(db, node, store_outputs=True, store_input_nodes_recursively=True)

# idb.store_node_outputs(pf.graph.nodes["IterNode"].node)

In [ ]:
type(db)

In [ ]:
Session = sessionmaker(bind=db.engine)
session = Session()

df = pd.read_sql(session.query(db.table).statement, session.bind)

df

In [ ]:
db.drop()
db.init()

In [ ]:
xx

In [ ]:
pf = gui.PyironFlow(['assyst', 'linearfit2', 'graph', 'executor', 'executor2', 'murnaghan', 'murnaghan2']) # 'landau2',
pf.gui

In [ ]:
pf = gui.PyironFlow(['assyst', 'calphy2', 'linearfit2', 'graph', 'executor', 'executor2', 'murnaghan', 'murnaghan2']) # 'landau2',
pf.gui

In [ ]:
import pyiron_workflow.simple_workflow as sw

label ="subgraph"
label = "Linspace"
pf.graph.nodes[label].node.copy()._graph_node == pf.graph.nodes[label]

In [ ]:
import pickle

new_node = pickle.loads(pickle.dumps(pf.graph.nodes["subgraph"]))
new_node.node

In [ ]:
base.GraphNode().__setstate__(pf.graph.nodes["subgraph"].__getstate__()).node.run()

In [ ]:
from pyiron_workflow import Workflow
import pyiron_nodes as pn

wf = Workflow("test")
wf.bulk = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.volume = pn.atomistic.structure.calc.Volume(wf.bulk)
# wf.subgraph = Group([wf.bulk, wf.volume])

wf.range = pn.math.Linspace(1.8, 2.2, 7)
# wf.executor = pn.executor.
# wf.iter = pn.executors.IterNode(node=wf.volume, kwarg_name="Bulk__name", kwarg_list=wf.range)

graph = base.get_graph_from_wf(wf, [], [])
graph_sub = base.create_group(graph, [0, 2])

base.get_updated_graph(graph_sub)
#graph_sub

In [ ]:
# graph.nodes["test"] = base.GraphNode()

In [ ]:
import json
from pyiron_workflow.graph.to_code import get_code_from_graph, _build_function_parameters

wf_file = "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))

# base.Nodes().__setstate__(graph_dict["nodes"])
sub_nodes= graph_dict["nodes"]["subgraph"]["graph"]["nodes"]
#base.Nodes().__setstate__(sub_nodes)
sub_nodes

In [ ]:
import json
from pyiron_workflow.graph.to_code import get_code_from_graph, _build_function_parameters

wf_file = "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))


    
 
# loaded_graph = base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"])
loaded_graph = base.Graph().__setstate__(graph_dict)
#print(get_code_from_graph(loaded_graph.nodes["subgraph"].graph))


# print(get_code_from_graph(loaded_graph.nodes["subgraph"]))
#loaded_graph.nodes["subgraph"].node

#base.graph_to_node(base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"]))
subgraph = base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"])

#get_code_from_graph(subgraph)
subgraph.nodes._obj_type is None
#subgraph

In [ ]:
graph_dict["nodes"]["subgraph"]["graph"]["nodes"]

In [ ]:
loaded_graph.nodes._obj_type

The decorator functions of Nodes are a constant source of errors. Can we make them more elegant and robust by going from decorator arguments to class inheritance?? Do I really need something like ob_type? I know the type I want to get from the name, i.e., nodes should have always GraphNode elements.

In [ ]:
subgraph = base._load_graph("murnaghan2").nodes["subgraph"]
print('xxx: ', base.graph_to_code(subgraph.graph))

pyiron_workflow.graph.gui.GuiGraph(subgraph.graph)

In [ ]:
wf_file = "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))

loaded_graph = base.Graph().__setstate__(graph_dict)

subgraph2 = loaded_graph.nodes["subgraph"].graph
print('xxx: ', base.graph_to_code(subgraph2))
pyiron_workflow.graph.gui.GuiGraph(subgraph2)

In [ ]:
from pyiron_workflow.graph.to_code import get_code_from_graph, _build_function_parameters


print(get_code_from_graph(loaded_graph.nodes["subgraph"].graph))

In [ ]:
subgraph = base._load_graph("murnaghan2").nodes["subgraph"]
print('xxx: ', base.graph_to_code(subgraph.graph))

In [ ]:
from pyiron_workflow.graph.to_code import get_code_from_graph, _build_function_parameters

_build_function_parameters(subgraph.graph, use_node_default=False)

In [ ]:
print('xxx: ', get_code_from_graph(subgraph.graph, use_node_default=False))

In [ ]:
from pyiron_workflow.graph.to_code import get_code_from_graph, _build_function_parameters

graph = base._load_graph("murnaghan2")
print(get_code_from_graph(graph.nodes["subgraph"].graph))

In [ ]:
base.Graph(id=None, 
      label='subgraph', 
      root_node=None, 
      nodes=NestedDict({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', 
                                          parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x17372c3e0>, 
                                          graph=None, node_type='node', widget_type='customNode', expanded=False), 
                        'M3GNet': GraphNode(id='M3GNet', import_path='pyiron_nodes.atomistic.engine.ase.M3GNet', label='M3GNet', 
                                            parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x17372c410>, 
                                            graph=None, node_type='node', widget_type='customNode', expanded=False), 
                        'Static': GraphNode(id='Static', import_path='pyiron_nodes.atomistic.calculator.ase.Static', label='Static', 
                                            parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x17372c0e0>, 
                                            graph=None, node_type='node', widget_type='customNode', expanded=False), 
                        'GetEnergyLast': GraphNode(id='GetEnergyLast', import_path='pyiron_nodes.atomistic.calculator.output.GetEnergyLast', label='GetEnergyLast', 
                                                   parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x17372c3b0>, 
                                                   graph=None, node_type='node', widget_type='customNode', expanded=False)}), 
      edges=[GraphEdge(source='Static', target='GetEnergyLast', sourceHandle='out', targetHandle='calculator'), 
             GraphEdge(source='Bulk', target='Static', sourceHandle='structure', targetHandle='structure'), 
             GraphEdge(source='M3GNet', target='Static', sourceHandle='out', targetHandle='engine')], graph={})

In [ ]:
subgraph.graph

In [ ]:
subgraph.node.inputs

In [ ]:
graph2 = base._load_graph("executor3")
for node in graph2.nodes.values():
    print(node)

In [ ]:
%%time

from pyiron_workflow import simple_workflow as sw
import pyiron_nodes
Al = pyiron_nodes.atomistic.structure.build.CubicBulkCell(element="Al")

from pyiron_workflow.graph import base
import pickle
dumps = pickle.dumps((Al, base.run_node))

Al.__getstate__()
Al_p, run = pickle.loads(dumps)
Al_p.inputs
# Al
base.Node().__setstate__(state=Al.__getstate__())

In [ ]:
graph = base._load_graph("executor3")
subgraph = graph.nodes["subgraph"].graph
#base.graph_to_node(subgraph)
for node in subgraph.nodes.values():
    g = base.GraphNode().__setstate__(node)
    print('NODE: ', g.label)

In [ ]:
graph = base._load_graph("executor3")
subgraph = graph.nodes["subgraph"].graph
#base.graph_to_node(subgraph)
for node in subgraph.nodes.values():
    g = base.GraphNode().__setstate__(node)
    print('NODE: ', g.label)

In [ ]:
from pyiron_nodes.atomistic.structure.build import Bulk

In [ ]:
import importlib

module = importlib.import_module('pyiron_nodes.atomistic.structure.build')
module.Bulk()

In [ ]:
sw.get_node_from_path('pyiron_nodes.atomistic.structure.build.Bulk')()

In [ ]:
state = pickle.dumps(Al)
state

In [ ]:
import copy

copy.copy(Al_p).inputs

In [ ]:
graph = base._load_graph("bulk")
# base._nodes_to_gui(graph)
#graph.nodes #["Bulk"]
graph

In [ ]:
node = base.Node().__setstate__(state=Al.__getstate__())
node.inputs

In [ ]:
pf = gui.PyironFlow(['assyst', 'calphy2', 'linearfit2', 'graph', 'executor', 'executor3']) # 'landau2',
pf.gui

In [ ]:
xx

In [ ]:
graph = pf.graph #.__getstate__()
# base._get_subgraph(graph, [0,1]).nodes.__getstate__()
graph.nodes.iloc(1).__getstate__()

In [ ]:
pf.graph.nodes["subgraph"].__getstate__()

In [ ]:
base.graph_to_node(pf.graph.nodes["subgraph"].graph)

In [ ]:
base._load_graph("executor3")

In [ ]:
state = pf.graph.nodes["Bulk"].node.__getstate__()
print(state)
base.Node().__setstate__(state)

In [ ]:
dumps = pickle.dumps(pf.graph.nodes)

In [ ]:
pickle.loads(dumps)

In [ ]:
pf.graph.nodes

In [ ]:
pf.graph.nodes["Bulk"]

In [ ]:
node = pf.graph.nodes["subgraph"]
node.node_type

In [ ]:
base.run_macro_node(node.node)

In [ ]:
subgraph = base.get_graph_from_macro_node(node.node)
subgraph.nodes

I need a way to describe/include virtual nodes both in the code as in the graph representation. Macros do not help since they are given explicitly as a file.
- store original graph + grouping commands
    - makes grouping/ungrouping very easy
    - natural extension of expand/collaps
    - we would then have nodes, edges + groups
    - update would then not only handle expand but also grouping
    - storage etc. would work as before (ideally no extra implementation)
    - issue: how to translate this into code (add virtual nodes as macro to code)
- add macro definitions to code
    - assumption: we do not want to describe all groupings performed by the user, i.e., virtual nodes make sense and are needed
    - pros: human readable, easy to put them later if needed to file
    - question: how does it work together with with run_macro function etc. 

In [ ]:
from pyiron_workflow.graph import base

print(base.graph_to_code(pf.graph))

In [ ]:
node.node_type, node.label

In [ ]:
node._func()

In [ ]:
from pyiron_workflow.graph import base

base.run_macro_node(node)

In [ ]:
pf.graph.nodes["IterNode"].node()

In [ ]:
from pyiron_nodes.atomistic.structure.build import Bulk

Bulk()._func(name='Al')

In [ ]:
bulk = Bulk()
bulk.inputs["name"] = "Al"
bulk.inputs

In [ ]:
Bulk()('Al', a=2)

In [ ]:
list(set(['Volume', 'Bulk', 'Volume']))

In [ ]:
from pyiron_workflow.graph import base

selected_nodes = ["Bulk", "Volume"]
node_ids = base._node_labels_to_node_ids(pf.graph, selected_nodes)

base.create_group(pf.graph, node_ids).nodes["subgraph"].node._workflow # .run()  # graph.nodes["Bulk"].node.run()

In [ ]:
import inspect

f = base.create_group(pf.graph, node_ids).nodes["subgraph"].graph  #.graph.nodes #.nodes["Bulk"].node.inputs
print(base.graph_to_code(f))

In [ ]:
f.nodes

In [ ]:
base.create_group(pf.graph, node_ids)#.nodes["subgraph"].nodes

In [ ]:
selected_nodes = ["Bulk", "Volume"]
node_ids = base._node_labels_to_node_ids(pf.graph, selected_nodes)

print(get_code_from_graph1(pf.graph))

In [ ]:
print(get_code_from_graph(graph, use_node_default=False))

In [ ]:
from pyiron_workflow import as_macro_node

@as_macro_node("out")
def subgraph(Bulk__name: str = "Al"):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Bulk = pyiron_nodes.atomistic.structure.build.Bulk(name=Bulk__name)
    wf.Volume = pyiron_nodes.atomistic.structure.calc.Volume(structure=wf.Bulk)

    return wf.Volume

subgraph()(Bulk__name="Al")

In [ ]:
subgraph().node_type

In [ ]:
graph = pf.graph
graph = base.topological_sort(graph)

base.get_non_default_input(graph, flatten=True)

In [ ]:
import importlib

import importlib
from pyiron_workflow.graph import to_code

# Reload the module
importlib.reload(to_code)

from pyiron_workflow.graph.to_code import get_code_from_graph

print(get_code_from_graph(f, use_node_default=False))

In [ ]:
import black

black.format_str()

In [ ]:
mode = black.Mode()

print(black.format_str(get_code_from_graph(pf.graph), mode=mode))

In [ ]:
import pyiron_nodes


wf = Workflow('calphy2')

element = "Al"
cell_size = 3
solid_fraction = 0

wf.CubicBulkCell = pyiron_nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size) 
wf.Potential = pyiron_nodes.atomistic.engine.lammps.Potential(structure=wf.CubicBulkCell) 
wf.Tolerance = pyiron_nodes.atomistic.property.calphy.Tolerance(solid_fraction=solid_fraction) 
wf.InputClass = pyiron_nodes.atomistic.property.calphy.InputClass() 
wf.InputClass_1 = pyiron_nodes.atomistic.property.calphy.InputClass(tolerance=wf.Tolerance) 
wf.SolidFreeEnergyWithTemperature = pyiron_nodes.atomistic.property.calphy.SolidFreeEnergyWithTemperature(structure=wf.CubicBulkCell, potential=wf.Potential, inp=wf.InputClass_1, store=False) 
wf.LiquidFreeEnergyWithTemperature = pyiron_nodes.atomistic.property.calphy.LiquidFreeEnergyWithTemperature(potential=wf.Potential, structure=wf.CubicBulkCell, inp=wf.InputClass, store=False) 
wf.CalculatePhaseTransformationTemperature = pyiron_nodes.atomistic.property.calphy.CalculatePhaseTransformationTemperature(t1=wf.SolidFreeEnergyWithTemperature.outputs.t, 
                                                                                                                            f1=wf.SolidFreeEnergyWithTemperature.outputs.f, 
                                                                                                                            t2=wf.LiquidFreeEnergyWithTemperature.outputs.t, 
                                                                                                                            f2=wf.LiquidFreeEnergyWithTemperature.outputs.f) 

wf.CalculatePhaseTransformationTemperature.pull()

In [ ]:
from pyiron_workflow.graph.base import get_full_graph_from_wf, get_updated_graph, pull_node, _load_graph

In [ ]:
# graph = full_graph = get_full_graph_from_wf(wf)
graph = _load_graph("calphy2.json")
pull_node(get_updated_graph(graph), "LiquidFreeEnergyWithTemperature")

critical feature:
- executorlib must be able to run single node or (sub)-workflow
- executorlib is a task manager (not a workflow executor)
- library stack:
    - workflow_creator
    - workflow_executor
    - executor_lib
- application:
    - exec.submit(node)  

In [ ]:
from pyiron_workflow import as_macro_node


@as_macro_node("transition_temperature")
def calphy2(element: str, cell_size: int = 1, solid_fraction: float = 0.7):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('calphy2')

    wf.CubicBulkCell = pyiron_nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size) 
    wf.Tolerance = pyiron_nodes.atomistic.property.calphy.Tolerance(solid_fraction=solid_fraction) 
    wf.InputClass = pyiron_nodes.atomistic.property.calphy.InputClass() 
    wf.Potential = pyiron_nodes.atomistic.engine.lammps.Potential(structure=wf.CubicBulkCell) 
    wf.InputClass_1 = pyiron_nodes.atomistic.property.calphy.InputClass(tolerance=wf.Tolerance) 
    wf.LiquidFreeEnergyWithTemperature = pyiron_nodes.atomistic.property.calphy.LiquidFreeEnergyWithTemperature(potential=wf.Potential, structure=wf.CubicBulkCell, inp=wf.InputClass) 
    wf.SolidFreeEnergyWithTemperature = pyiron_nodes.atomistic.property.calphy.SolidFreeEnergyWithTemperature(structure=wf.CubicBulkCell, potential=wf.Potential, inp=wf.InputClass_1) 
    wf.CalculatePhaseTransformationTemperature = pyiron_nodes.atomistic.property.calphy.CalculatePhaseTransformationTemperature(t1=wf.SolidFreeEnergyWithTemperature.outputs.t, f1=wf.SolidFreeEnergyWithTemperature.outputs.f, t2=wf.LiquidFreeEnergyWithTemperature.outputs.t, f2=wf.LiquidFreeEnergyWithTemperature.outputs.f) 

    return wf.CalculatePhaseTransformationTemperature# .outputs.transition_temp #, wf.CalculatePhaseTransformationTemperature.outputs.plot

    

It appears that macro of a macro does not work
- check!!

In [ ]:
@as_macro_node("transition_temperature")
def calphy2(element: str, cell_size: int = 1, solid_fraction: float = 0.7):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('calphy2')

    # wf.CubicBulkCell = pyiron_nodes.atomistic.structure.build.Bulk("Al") # CubicBulkCell(element=element, cell_size=cell_size) 
    wf.CubicBulkCell = pyiron_nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size)

    return wf.CubicBulkCell

In [ ]:
graph = base.Graph()
graph += pyiron_nodes.atomistic.structure.build.CubicBulkCell(element="Al")
graph.nodes

In [ ]:
from pyiron_workflow.graph import base

n = calphy2(element="Al")
graph = base.get_graph_from_macro_node(n)
# base.run_macro_node(n)
# base.GuiGraph(base.get_graph_from_macro_node(n))
# base.get_graph_from_macro_node(n).nodes["CubicBulkCell"].node.run()
graph.nodes

In [ ]:
import pyiron_nodes as pn

In [ ]:
from typing import Optional

@as_macro_node("structure")
def CubicBulkCell(
    element: str, cell_size: int = 1, vacancy_index: Optional[int] = None
):
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.structure.transform import (
        CreateVacancy,
        Repeat,
    )
    from pyiron_workflow import Workflow

    wf = Workflow('macro')

    wf.bulk = Bulk(name=element, cubic=True)
    wf.repeat = Repeat(structure=wf.bulk, repeat_scalar=cell_size)

    wf.vacancy = CreateVacancy(structure=wf.repeat, index=vacancy_index)
    return wf.vacancy

In [ ]:
CubicBulkCell("Al").run()

In [ ]:
pn.atomistic.structure.build.CubicBulkCell(element="Al")

In [ ]:
from executorlib import SingleNodeExecutor

In [ ]:
%%time
with SingleNodeExecutor(max_workers=4, block_allocation=True) as exe:
    future_lst = [exe.submit(sum, [i, i]) for i in range(2, 5)]
    print([f.result() for f in future_lst])

In [ ]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True




In [ ]:
%%time
with SingleNodeExecutor(max_workers=6, block_allocation=False) as exe:
    future_lst = [exe.submit(is_prime, n) for n in PRIMES]
    print([f.result() for f in future_lst])

In [ ]:
%%time
with SingleNodeExecutor(max_workers=1, block_allocation=True) as exe:
    future_lst = [exe.submit(pn.atomistic.structure.build.Bulk()._func, n) for n in ["Al", "Fe", "Ni", "Zn"]]
    print([f.result() for f in future_lst])

In [ ]:
Al = pn.atomistic.structure.build.Bulk()._func('Al')
Al.species[0].name

In [ ]:
%%time
for n in ["Al", "Fe"]:
    print(pn.atomistic.structure.build.Bulk(n).run())

In [ ]:
import pyiron_nodes as pn
import concurrent.futures
import urllib.request

elements = ['Al', 'Fe']


# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    # Start the load operations and mark each future with its URL
    futures = {executor.submit(pn.atomistic.structure.build.Bulk()._func, el, **{'cubic': True}): el for el in elements}
    for future in concurrent.futures.as_completed(futures):
        out = futures[future]
        print("out: ", out)

        data = future.result()
        print(data.__repr__())

In [ ]:
from pyiron_workflow import as_function_node, Node

@as_function_node
def IterNode(node: Node, kwarg_name: str, kwarg_list: list, max_workers: int=1, executor=None):
    # from concurrent.futures import ThreadPoolExecutor as Executor
    from executorlib import SingleNodeExecutor as Executor
    from concurrent.futures import as_completed

    out_dict = dict()
    with Executor(max_workers=max_workers) as executor:
        # Start the load operations and mark each future with its index
        futures = {executor.submit(node._func, **{kwarg_name: el}): el for el in kwarg_list}
        for future in as_completed(futures):
            out = futures[future]    
            data = future.result()
            out_dict[out] = data.species[0].name
            
            # print("out: ", out)
            # print(data.__repr__())
            
    return out_dict
    

In [ ]:
%%time
elements = ['Al', 'Fe']

IterNode(pn.atomistic.structure.build.Bulk(), "name", elements, max_workers=2).run()

In [ ]:
import numpy as np

# Define the matrix mat and the index array ind1
mat = np.array([[0, 1, 2],
                [3, 4, 5],
                [6, 7, 8]])

ind1 = np.array([1, 0, 2])

# Compute v using advanced indexing
v = mat[ind1, np.arange(len(ind1))]

v  # Output: [3, 1, 8]

In [ ]:
import pyiron_nodes as pn

n = pn.atomistic.structure.build.Bulk("Al")

In [ ]:
kwargs = n.kwargs
kwargs.update({"a": 2.1})
kwargs